_______________________________________
#### 1° Prueba

In [ ]:
import os

# Fix env windows
os.environ["HF_HUB_DISABLE_SYMLINKS"] = "1"
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
os.environ["SPEECHBRAIN_CACHE_STRATEGY"] = "copy"

from dotenv import load_dotenv
from pydub import AudioSegment
from pyannote.audio import Pipeline

import ipywidgets as widgets
widgets.IntSlider()

# import warnings
# warnings.filterwarnings(
#     "ignore",
#     message=".*deprecated.*"
# )

# Load environment variables from .env
load_dotenv()

# Hugging Face Token
HF_TOKEN = os.getenv("PYANNOTE_HF_TOKEN")
assert HF_TOKEN, "PYANNOTE_HF_TOKEN not defined."

In [ ]:
import torch
x = torch.rand(5, 3)
print(x)

In [ ]:
INPUT_DIR = "../audios"
OUTPUT_DIR = "../output"
PRE_CUT_SECONDS = 1.5  # segundos antes de que empiece el niño
MIN_CHILD_SEGMENT = 1.0  # duración mínima para considerar "voz del niño"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
pipeline = Pipeline.from_pretrained(
   'pyannote/speaker-diarization-3.0',
   use_auth_token=HF_TOKEN
)

config.yaml:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

speaker-embedding.onnx:   0%|          | 0.00/26.5M [00:00<?, ?B/s]

In [ ]:
# ==========================
# FUNCIÓN PRINCIPAL
# ==========================
def process_audio(audio_path):
    print(f"Procesando: {os.path.basename(audio_path)}")

    # Cargar audio
    audio = AudioSegment.from_wav(audio_path)
    duration = len(audio) / 1000  # segundos

    # Diarización
    diarization = pipeline(audio_path)

    # Extraer segmentos ordenados
    segments = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        segments.append({
            "speaker": speaker,
            "start": turn.start,
            "end": turn.end,
            "duration": turn.end - turn.start
        })

    segments.sort(key=lambda x: x["start"])

    if len(segments) < 2:
        print("⚠️ No se detectaron suficientes hablantes")
        return None

    # ==========================
    # ASUMIMOS:
    # - Primer hablante = profesor
    # - Segundo hablante largo = niño
    # ==========================
    first_speaker = segments[0]["speaker"]

    child_start = None
    for seg in segments:
        if seg["speaker"] != first_speaker and seg["duration"] >= MIN_CHILD_SEGMENT:
            child_start = seg["start"]
            break

    if child_start is None:
        print("⚠️ No se detectó inicio del niño")
        return None

    # Aplicar margen
    cut_time = max(0, child_start - PRE_CUT_SECONDS)

    # Cortar audio
    cut_audio = audio[int(cut_time * 1000):]

    return cut_audio, cut_time

In [ ]:
# ==========================
# PROCESAR CARPETA
# ==========================
for file in os.listdir(INPUT_DIR):
    if not file.lower().endswith(".wav"):
        continue

    input_path = os.path.join(INPUT_DIR, file)
    output_path = os.path.join(
        OUTPUT_DIR,
        file.replace(".wav", "_cut.wav")
    )

    result = process_audio(input_path)

    if result is None:
        continue

    cut_audio, cut_time = result
    cut_audio.export(output_path, format="wav")

    print(f"✔ Guardado: {output_path} (corte en {cut_time:.2f}s)")


Procesando: TA10005.wav
⚠️ No se detectó inicio del niño
Procesando: TA10014.wav
⚠️ No se detectó inicio del niño
Procesando: TA10033.wav
✔ Guardado: ../output\TA10033_cut.wav (corte en 9.05s)
Procesando: TA40171.wav
⚠️ No se detectó inicio del niño
Procesando: TA40173.wav
✔ Guardado: ../output\TA40173_cut.wav (corte en 5.42s)
Procesando: TA40176.wav
⚠️ No se detectó inicio del niño


_________________
##### 2° Prueba
_________________

In [ ]:
import os

# Fix env windows
# os.environ["SB_DISABLE_SYMLINKS"] = "1"
# os.environ["HF_HUB_DISABLE_SYMLINKS"] = "1"
# os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
# os.environ["SPEECHBRAIN_CACHE_STRATEGY"] = "copy"

PROJECT_CACHE = os.path.join(os.getcwd(), ".torch_cache")

os.environ["TORCH_HOME"] = PROJECT_CACHE
os.environ["SB_DISABLE_SYMLINKS"] = "1"
os.environ["SPEECHBRAIN_CACHE_STRATEGY"] = "copy"
os.environ["HF_HOME"] = os.path.join(os.getcwd(), ".hf_cache")

# Load environment variables from .env
from dotenv import load_dotenv
load_dotenv()

# Main order to garanty pyannote functuionality
from pyannote.audio import Pipeline
from pydub import AudioSegment
from pydub.effects import normalize
from pydub.silence import detect_silence

# Widgets
# import ipywidgets as widgets
# widgets.IntSlider()

# import warnings
# warnings.filterwarnings(
#     "ignore",
#     message=".*deprecated.*"
# )

# Check token Hugging Face
HF_TOKEN = os.getenv("PYANNOTE_HF_TOKEN")
assert HF_TOKEN, "PYANNOTE_HF_TOKEN not defined."

c:\Users\Denisse Orellana\Desktop\editar_audio\venv_py3.10\lib\site-packages\pyannote\audio\core\io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [ ]:
# =========================
# CONFIGURACIÓN
# =========================

MIN_ACCUMULATED_CHILD_TIME = 1.5  # segundos acumulados del niño
PRE_CUT_SECONDS = 0.8             # margen antes del inicio del niño
MIN_SILENCE_LEN = 700             # ms
SILENCE_THRESH = -40              # dBFS

In [ ]:
# =========================
# UTILIDADES
# =========================

def load_and_normalize_audio(path):
    audio = AudioSegment.from_wav(path)
    return normalize(audio)


def diarize_audio(audio_path):
    pipeline = Pipeline.from_pretrained(
        "pyannote/speaker-diarization",
        use_auth_token=HF_TOKEN
    )
    return pipeline(audio_path)


def diarization_to_segments(diarization):
    segments = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        segments.append({
            "speaker": speaker,
            "start": turn.start,
            "end": turn.end,
            "duration": turn.end - turn.start
        })
    return segments


def detect_child_start_by_diarization(segments):
    if not segments:
        return None

    first_speaker = segments[0]["speaker"]
    acc_time = 0.0
    start_time = None

    for seg in segments:
        if seg["speaker"] != first_speaker:
            if start_time is None:
                start_time = seg["start"]
            acc_time += seg["duration"]

            if acc_time >= MIN_ACCUMULATED_CHILD_TIME:
                return start_time
        else:
            acc_time = 0.0
            start_time = None

    return None


def detect_child_start_by_silence(audio):
    silences = detect_silence(
        audio,
        min_silence_len=MIN_SILENCE_LEN,
        silence_thresh=SILENCE_THRESH
    )

    if not silences:
        return None

    # Tomamos el primer silencio largo
    first_silence_end = silences[0][1]
    return first_silence_end / 1000.0


In [ ]:
# =========================
# FUNCIÓN PRINCIPAL
# =========================

def extract_child_audio(input_wav, output_wav):
    print(f"\n🎧 Procesando: {os.path.basename(input_wav)}")

    audio = load_and_normalize_audio(input_wav)

    # 1️⃣ Diarización
    try:
        diarization = diarize_audio(input_wav)
        segments = diarization_to_segments(diarization)
    except Exception as e:
        print(f"⚠️ Error en diarización: {e}")
        segments = []

    # 2️⃣ Detección por hablante
    child_start = detect_child_start_by_diarization(segments)

    # 3️⃣ Fallback por silencio
    if child_start is None:
        print("⚠️ Fallback: usando detección por silencio")
        child_start = detect_child_start_by_silence(audio)

    if child_start is None:
        print("❌ No se pudo detectar inicio del niño")
        return False

    cut_time = max(0, child_start - PRE_CUT_SECONDS)
    print(f"✅ Inicio niño detectado en {child_start:.2f}s (corte en {cut_time:.2f}s)")

    child_audio = audio[int(cut_time * 1000):]
    child_audio.export(output_wav, format="wav")

    print(f"💾 Audio guardado: {output_wav}")
    return True



In [ ]:
INPUT_FOLDER = "../audios"
OUTPUT_FOLDER = "../output"

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
for file in os.listdir(INPUT_FOLDER):
    if file.lower().endswith(".wav"):
        in_path = os.path.join(INPUT_FOLDER, file)
        out_path = os.path.join(OUTPUT_FOLDER, file)
        extract_child_audio(in_path, out_path)


🎧 Procesando: TA10005.wav


config.yaml:   0%|          | 0.00/500 [00:00<?, ?B/s]

c:\Users\Denisse Orellana\Desktop\editar_audio\venv_py3.10\lib\site-packages\pyannote\audio\pipelines\speaker_verification.py:43: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  backend = torchaudio.get_audio_backend()
c:\Users\Denisse Orellana\Desktop\editar_audio\venv_py3.10\lib\site-packages\pyannote\audio\pipelines\speaker_verification.py:45: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import (
c:\Users\Denisse Orellana\Desktop\editar_audio\venv_py3.10\lib\site-packages\pyannote\audio\pipelines\speaker_verification.py:53: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove th

pytorch_model.bin:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/318 [00:00<?, ?B/s]

...

Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.2+cpu. Bad things might happen unless you revert torch to 1.x.
⚠️ Error en diarización: [WinError 1314] El cliente no dispone de un privilegio requerido: 'c:\\Users\\Denisse Orellana\\Desktop\\editar_audio\\notebooks\\.hf_cache\\hub\\models--speechbrain--spkrec-ecapa-voxceleb\\snapshots\\0f99f2d0ebe89ac095bcc5903c4dd8f72b367286\\hyperparams.yaml' -> 'C:\\Users\\Denisse Orellana\\.cache\\torch\\pyannote\\speechbrain\\hyperparams.yaml'
⚠️ Fallback: usando detección por silencio
❌ No se pudo detectar inicio del niño

🎧 Procesando: TA40173.wav


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\Denisse Orellana\.cache\torch\pyannote\models--pyannote--segmentation\snapshots\c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b\pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.2+cpu. Bad things might happen unless you revert torch to 1.x.
⚠️ Error en diarización: [WinError 1314] El cliente no dispone de un privilegio requerido: 'c:\\Users\\Denisse Orellana\\Desktop\\editar_audio\\notebooks\\.hf_cache\\hub\\models--speechbrain--spkrec-ecapa-voxceleb\\snapshots\\0f99f2d0ebe89ac095bcc5903c4dd8f72b367286\\hyperparams.yaml' -> 'C:\\Users\\Denisse Orellana\\.cache\\torch\\pyannote\\speechbrain\\hyperparams.yaml'
⚠️ Fallback: usando detección por silencio
✅ Inicio niño detectado en 0.99s (corte en 0.19s)
💾 Audio guardado: ../output\TA40173.wav

🎧 Procesando: TA40176.wav


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.6.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\Denisse Orellana\.cache\torch\pyannote\models--pyannote--segmentation\snapshots\c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b\pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.2+cpu. Bad things might happen unless you revert torch to 1.x.
⚠️ Error en diarización: [WinError 1314] El cliente no dispone de un privilegio requerido: 'c:\\Users\\Denisse Orellana\\Desktop\\editar_audio\\notebooks\\.hf_cache\\hub\\models--speechbrain--spkrec-ecapa-voxceleb\\snapshots\\0f99f2d0ebe89ac095bcc5903c4dd8f72b367286\\hyperparams.yaml' -> 'C:\\Users\\Denisse Orellana\\.cache\\torch\\pyannote\\speechbrain\\hyperparams.yaml'
⚠️ Fallback: usando detección por silencio
❌ No se pudo detectar inicio del niño


___________________________________
#### 3° Prueba silencios 12-01-2025
Algoritmo para detectar silencios. No realiza diarización, ni segmentación.
* not working
___________________________________

In [12]:
from pydub import AudioSegment
from pydub.effects import normalize
from pydub.silence import detect_silence
import os


def extract_child_audio(
    audio_path,
    output_path,
    min_silence_len=700,      # ms de silencio real
    silence_thresh_offset=16, # dB bajo el promedio
    min_reading_duration=3.0, # segundos mínimos de lectura
    pre_cut_seconds=0.8       # margen antes de que empiece el niño
):
    """
    Detecta el inicio de la lectura del niño usando:
    - normalización
    - detección de silencio
    - habla sostenida
    """

    print(f"🎧 Procesando: {os.path.basename(audio_path)}")

    # =========================
    # 1. Cargar y normalizar
    # =========================
    audio = AudioSegment.from_wav(audio_path)
    audio = normalize(audio)

    duration_sec = len(audio) / 1000

    # =========================
    # 2. Detectar silencios
    # =========================
    silence_thresh = audio.dBFS - silence_thresh_offset

    silences = detect_silence(
        audio,
        min_silence_len=min_silence_len,
        silence_thresh=silence_thresh
    )

    if not silences:
        print("⚠️ No se detectaron silencios claros")
        return False

    # =========================
    # 3. Buscar inicio del niño
    # =========================
    child_start = None

    for silence_start, silence_end in silences:
        # El niño debería empezar DESPUÉS del primer silencio largo
        candidate_start = silence_end / 1000  # segundos

        remaining = duration_sec - candidate_start
        if remaining >= min_reading_duration:
            child_start = candidate_start
            break

    if child_start is None:
        print("❌ No se pudo detectar inicio del niño")
        return False

    # =========================
    # 4. Aplicar margen
    # =========================
    cut_time = max(0, child_start - pre_cut_seconds)

    child_audio = audio[int(cut_time * 1000):]

    # =========================
    # 5. Validar duración final
    # =========================
    final_duration = len(child_audio) / 1000
    if final_duration < min_reading_duration:
        print("❌ Segmento demasiado corto para ser lectura")
        return False

    # =========================
    # 6. Exportar
    # =========================
    child_audio.export(output_path, format="wav")

    print(f"✅ Niño detectado desde {cut_time:.2f}s → guardado")
    return True


In [13]:
INPUT_FOLDER = "../audios"
OUTPUT_FOLDER = "../output"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

for file in os.listdir(INPUT_FOLDER):
    if file.lower().endswith(".wav"):
        in_path = os.path.join(INPUT_FOLDER, file)
        out_path = os.path.join(OUTPUT_FOLDER, file)

        extract_child_audio(in_path, out_path)

🎧 Procesando: TA10005.wav
⚠️ No se detectaron silencios claros
🎧 Procesando: TA10014.wav
⚠️ No se detectaron silencios claros
🎧 Procesando: TA10033.wav
✅ Niño detectado desde 0.03s → guardado
🎧 Procesando: TA40171.wav
⚠️ No se detectaron silencios claros
🎧 Procesando: TA40173.wav
✅ Niño detectado desde 0.17s → guardado
🎧 Procesando: TA40176.wav
✅ Niño detectado desde 0.45s → guardado
